In [1]:
from genericpath import isfile
import os
import re
import pickle
import logging
import netCDF4
from calendar import monthrange
from tqdm import tqdm
from glob import glob
from typing import Iterable
from datetime import datetime, timedelta, date
import numpy as np
import pandas as pd
import xarray as xr
# import xesmf as xe
from argparse import ArgumentParser
import h5py
import sys
sys.path.insert(1, "../")

# logger = logging.getLogger(__file__) this is for python scripts
##🚩 update this for jupyter notebook
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


In [2]:
import netCDF4 as nc

In [3]:
import os
print(os.getcwd())

/home/melvin/Documents/Melvin/AIms project /downscaling-cgan/notebooks


In [4]:
import os
import yaml

# Get the current working directory (where the notebook is running)
print(os.getcwd())
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(PROJECT_ROOT)

fcstyaml_path = os.path.join(PROJECT_ROOT, "config", "forecast_gfs.yaml")

# Open and parse the YAML file
with open(fcstyaml_path, "r") as f:
    fcst_params = yaml.safe_load(f)

print(fcst_params)


/home/melvin/Documents/Melvin/AIms project /downscaling-cgan/notebooks
/home/melvin/Documents/Melvin/AIms project /downscaling-cgan
{'MODEL': {'folder': 'gs://melvin_aims_bucket/cgan5/_cgan_5/', 'checkpoint': 36000}, 'INPUT': {'folder': 'gs://melvin_aims_bucket/NCGM/', 'dates': ['2024-04-20'], 'start_hour': 30, 'end_hour': 54}, 'OUTPUT': {'folder': 'gs://melvin_aims_bucket/cgan5/_cgan_5/forecasts/', 'ensemble_members': 50}}


In [5]:

#run this first in the terminal to authenticate with GCP
# gcloud auth application-default login 


In [7]:
!pip install tensorflow

  Using cached tensorflow-2.20.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
Using cached tensorflow-2.20.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (620.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tensorflow]2 [tensorflow]


In [1]:
import os

# Only needed for tensorflow versions greater than 2.16.0
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import sys
sys.path.insert(1,"../scripts/")
import forecast_gfs
# import forecast/

2026-02-17 17:36:53.339132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Setting up GPU


2026-02-17 17:36:59.043044: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/melvin/miniconda3/envs/cgan_new/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


{'EVAL': {'add_postprocessing_noise': True, 'avg_pooling': True, 'max_pooling': True, 'num_batches': 256, 'postprocessing_noise_factor': '1e-3'}, 'architecture': 'forceconv', 'disable_tf32': False, 'discriminator': {'filters_disc': 256, 'learning_rate_disc': 1e-05, 'normalisation': None}, 'downsample': False, 'downscaling_factor': 1, 'downscaling_steps': [1], 'generator': {'filters_gen': 64, 'latent_variables': 1, 'learning_rate_gen': 1e-05, 'noise_channels': 4, 'normalisation': None, 'output_activation': 'softplus'}, 'gpu_mem_incr': True, 'log_folder': '/home/melvin_aims_ac_za/logs/cgan', 'mode': 'GAN', 'padding': 'reflect', 'problem_type': 'normal', 'train': {'CL_type': 'ensmeanMSE', 'batch_size': 2, 'content_loss_weight': 1000.0, 'crop_size': 256, 'ensemble_size': 8, 'img_chunk_width': 256, 'kl_weight': 1e-08, 'num_epochs': 5, 'num_samples': 40000, 'rotate': False, 'steps_per_checkpoint': 2000, 'training_range': ['200101', '201501'], 'training_ratio': 5, 'training_weights': [0.25, 0

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
import importlib
importlib.reload(forecast_gfs)
for

In [10]:
import gcsfs
import pickle
import xarray as xr

# Automatic connection using default credentials
fs = gcsfs.GCSFileSystem()

# Read a NetCDF file from Google Cloud Storage
with fs.open('gs://melvin_aims_bucket/2024/20240101_00.nc') as f_obj:
    ds = xr.open_dataset(f_obj, engine='h5netcdf')


In [11]:
ds

<xarray.Dataset> Size: 544kB
Dimensions:        (latitude: 384, longitude: 352, time: 1)
Coordinates:
  * latitude       (latitude) float32 2kB -13.65 -13.55 -13.45 ... 24.55 24.65
  * longitude      (longitude) float32 1kB 19.15 19.25 19.35 ... 54.15 54.25
  * time           (time) datetime64[ns] 8B 2024-01-01
Data variables:
    precipitation  (latitude, longitude) float32 541kB ...
Attributes:
    description:  6h average of IMERG V07 rainfall data in the ICPAC region

In [12]:
import importlib
importlib.reload(forecast_gfs)

# forecast_gfs.make_fcst()

NameError: name 'forecast_gfs' is not defined

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
dates = np.asarray(fcst_params["INPUT"]["dates"],dtype='datetime64[D]')
d = dates[0].astype(object)
path_to_file = fcst_params["OUTPUT"]["folder"]+f"test/{d.year}/GAN_{d.year}{d.month:02}{d.day:02}.nc"

ds = xr.open_dataset(path_to_file)


fig = plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=1)
c = plt.pcolormesh(ds.longitude.values, ds.latitude.values, np.squeeze(ds.precipitation.mean(['valid_time','member']).values*24),
                   transform=ccrs.PlateCarree(), cmap='YlGnBu',vmin=1,vmax=60)

cb = plt.colorbar(c, fraction=0.02)